In [1]:
import meshcat
from utils import *
from interface import ExoSkeletonUDPInterface
import time 
import collections


# from estimation_problem import solve_estimation_problem
from utils import ArmMeasurementCurrent
from arm_model import create_arm
from pinocchio.visualize import MeshcatVisualizer as Visualizer


In [2]:
interface = ExoSkeletonUDPInterface()
counter = 0
interface.calibrate()

calibration ...
setting offset ...
finished calibration ...


In [2]:
rmodel, rdata, gmodel, cmodel = create_arm()
viz = Visualizer(rmodel, gmodel, cmodel)
viewer = meshcat.Visualizer(zmq_url = "tcp://127.0.0.1:6014")
viz.initViewer(viewer)
viz.loadViewerModel()
viz.initializeFrames()
viz.display_frames = True

add_frame("hand", viz.viewer)
add_frame("shoulder", viz.viewer)

# viz.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7014/static/


In [9]:
rmodel

Nb joints = 4 (nq=5,nv=5)
  Joint 0 universe: parent=0
  Joint 1 Shoulder: parent=0
  Joint 2 Elbow: parent=1
  Joint 3 lower_arm_rotation: parent=2

In [8]:
viz.display(np.array([0,np.pi/4.0,0,np.pi/2.0,0]))

In [39]:
names = ['hand', 'base', 'shoulder']
data_offset = []
offset = [pin.utils.rpyToMatrix(np.pi/2.0, 0, 0), 0]
counter = 0
T = 10
estimate_x0 = np.zeros(rmodel.nq + rmodel.nv)

measurement = collections.deque(maxlen=T)
offset_shoulder = 0
while True:
    interface.setCommand([0], [0.], [0], [0], [2.5])
    time.sleep(0.002)
    state = interface.getState()
    # print(state["base_acc"], state["shoulder_acc"], state["wrist_acc"])
    base = Rotation.from_quat(state["base_ori"][0]).as_matrix()
    shoulder = Rotation.from_quat(state["shoulder_ori"][0]).as_matrix()
    hand = Rotation.from_quat(state["wrist_ori"][0]).as_matrix()
    if counter < 100:
        data_offset.append([base.T @ shoulder, base.T @ hand])
    elif counter == 100:
        offset[0] = offset[0] @ data_offset[50][0].T
        offset[1] = data_offset[50][1].T
    else:
        update_frame('hand', viz.viewer, offset[1] @ base.T @ hand, [0.5, 0, 0])
        update_frame("shoulder", viz.viewer,   offset[0] @ base.T @ shoulder)
        measurement.append(ArmMeasurementCurrent( offset[0] @ base.T @ shoulder, offset[1] @ base.T @ hand))
        if counter > T + 100:
            st = time.time()
            estimate = solve_estimation_problem(measurement, T, rmodel, estimate_x0)
            
            if counter > 100 and counter < 120:
                offset_shoulder = state["q"] + estimate.xs[-1][1] 
            else:
                estimate.xs[-1][1] += offset_shoulder
            viz.display(estimate.xs[-1][:rmodel.nq])
            estimate_x0 = estimate.xs[-1]
            state["q"] -= offset_shoulder
            print((180/np.pi)*(state["q"]), (180/np.pi)*estimate_x0[1], (180/np.pi)*(state["q"] + estimate_x0[1]) )
    

    counter += 1


[89.47565996] -89.4756599616266 [0.]
[89.88139898] -89.88139898021157 [0.]
[89.77284034] -89.77284033685697 [0.]
[89.77653288] -89.77653287505245 [0.]
[89.78272878] -89.7827287830325 [0.]
[89.78295399] -89.78295398987262 [0.]
[89.79846345] -89.79846345235623 [0.]
[89.79860981] -89.7986098121761 [0.]
[89.798675] -89.79867499726429 [0.]
[89.798675] -89.81638938453422 [-0.01771439]


/tmp/ipykernel_262720/3089177220.py:34: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  estimate.xs[-1][1] += offset_shoulder


[89.798675] -89.78453071186698 [0.01414429]
[89.798675] -89.78333365855644 [0.01534134]
[89.798675] -89.49933095609191 [0.29934404]
[89.798675] -89.50134421454159 [0.29733078]
[89.798675] -89.51027613319424 [0.28839886]
[89.798675] -89.5095303977039 [0.2891446]
[89.798675] -89.50867954771341 [0.28999545]
[89.798675] -89.50231723100126 [0.29635777]
[89.80637945] -89.4959685720604 [0.31041088]
[89.80637945] -89.48467829780438 [0.32170115]
[89.80637945] -89.48102421414285 [0.32535524]
[89.80637945] -89.47679665601954 [0.32958279]
[89.80637945] -89.47174688482711 [0.33463257]
[89.80637945] -89.46167720637506 [0.34470224]
[89.80637945] -89.46024545261338 [0.346134]
[89.80637945] -89.46049294824132 [0.3458865]
[89.80637945] -89.45513375018697 [0.3512457]
[89.80637945] -89.45345911584155 [0.35292033]
[89.80637945] -89.4516535598891 [0.35472589]
[89.80637945] -89.45143178933243 [0.35494766]
[89.80637945] -89.44941927248223 [0.35696018]
[89.80637945] -89.44895774012008 [0.35742171]
[89.80637945

KeyboardInterrupt: 

In [17]:
import crocoddyl
from mim_solvers import SolverSQP
import numpy as np

def solve_estimation_problem(measurements, T, rmodel, x0):
    rdata = rmodel.createData()
    nq = rmodel.nq; nv = rmodel.nv; nu = nq; nx = nq+nv

    
    dt = 1e-2
    
    # # # # # # # # # # # # # # #
    ###  SETUP CROCODDYL OCP  ###
    # # # # # # # # # # # # # # #
    
    # State and actuation model
    state = crocoddyl.StateMultibody(rmodel)
    actuation = crocoddyl.ActuationModelFull(state)
    
    # Create cost terms 
    # Control regularization cost
    uResidual = crocoddyl.ResidualModelControlGrav(state)
    uRegCost = crocoddyl.CostModelResidual(state, uResidual)
    # State regularization cost
    xResidual = crocoddyl.ResidualModelState(state, x0)
    xRegCost = crocoddyl.CostModelResidual(state, xResidual)

    runningModel = []
    for i in range(T):
        # endeff frame orientation cost
        endeff_frame_id = rmodel.getFrameId("Hand")
        frameOrientationResidual = crocoddyl.ResidualModelFrameRotation(state, endeff_frame_id, measurements[i].palm_orientation)
        frameOrientationCost = crocoddyl.CostModelResidual(state, frameOrientationResidual)

        imu_arm_id = rmodel.getFrameId("imu_arm")
        imuArmOrientationResidual = crocoddyl.ResidualModelFrameRotation(state, imu_arm_id, measurements[i].arm_orientation)
        imuArmOrientationCost = crocoddyl.CostModelResidual(state, imuArmOrientationResidual)
        
        # Running and terminal cost models
        runningCostModel = crocoddyl.CostModelSum(state)
        terminalCostModel = crocoddyl.CostModelSum(state)
        
        # Add costs
        runningCostModel.addCost("stateReg", xRegCost, 5e-3)
        runningCostModel.addCost("ctrlRegGrav", uRegCost, 5e-3)
        runningCostModel.addCost("shoulderOrientation", imuArmOrientationCost, 5e2)
        runningCostModel.addCost("wristOrientation", frameOrientationCost, 6e2)
        
    
        # Create Differential Action Model (DAM), i.e. continuous dynamics and cost functions
        running_DAM = crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuation, runningCostModel)
        runningModel.append(crocoddyl.IntegratedActionModelEuler(running_DAM, dt))

    terminal_DAM = crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuation, runningCostModel)
    
    # Create Integrated Action Model (IAM), i.e. Euler integration of continuous dynamics and cost
    
    terminalModel = crocoddyl.IntegratedActionModelEuler(terminal_DAM, 0.)
        
    # Create the shooting problem
    problem = crocoddyl.ShootingProblem(x0, runningModel, terminalModel)
    
    # Create solver + callbacks
    ddp = SolverSQP(problem)
    # ddp = crocoddyl.SolverDDP(problem)
    ddp.setCallbacks([crocoddyl.CallbackLogger()])
    ddp.use_filter_line_search = True
    
    # Warm start : initial state + gravity compensation
    xs_init = [x0 for i in range(T+1)]
    us_init = ddp.problem.quasiStatic(xs_init[:-1])
    
    # Solve
    ddp.solve(xs_init, us_init, maxiter=10)
    
    return ddp